In [ ]:
import rtsvg
import hdbscan
import numpy as np
import random
from math import cos, sin, pi
to_rad = lambda d: d*pi/180
rt = rtsvg.RACETrack()

In [ ]:
n   = 250
pts = np.array([random.random()*360.0 for i in range(n)])
pts = pts.reshape(-1,1)
clusterer = hdbscan.HDBSCAN()
clusterer.fit(pts)
w,h,r = 600, 600, 290
svg = [f'<svg x="0" y="0" width="{w}" height="{h}"><rect width="{w}" height="{h}" fill="#ffffff" stroke="#ffffff" />']
for i in range(len(pts)):
    _x_ = (w/2) + r * cos(to_rad(pts[i][0])) 
    _y_ = (h/2) + r * sin(to_rad(pts[i][0]))
    _color_ = rt.co_mgr.getColor(clusterer.labels_[i])
    svg.append(f'<circle cx="{_x_}" cy="{_y_}" r="3.5" fill="{_color_}" />')
clusterer.labels_
rt.tile([''.join(svg)+'</svg>'])

In [3]:
_df_ = clusterer.condensed_tree_.to_pandas()
parent_to_children, child_to_parent = {}, {}
for i in range(len(_df_)):
    _parent_, _child_, _child_size_ = _df_.iloc[i]['parent'], _df_.iloc[i]['child'], _df_.iloc[i]['child_size']
    # print(f'{_parent_} -> {_child_} ({_child_size_})')
    if _parent_ not in parent_to_children.keys(): parent_to_children[_parent_] = []
    parent_to_children[_parent_].append(_child_)
    if _child_ not in child_to_parent.keys(): child_to_parent[_child_] = []
    child_to_parent[_child_].append(_parent_)

In [ ]:
svg_ring_one = []
parents_seen = set()
for i in range(len(pts)):
    _parent_ = child_to_parent[i][0]
    parents_seen.add(_parent_)
for _parent_ in parents_seen:
    all_children_at_leaves = True
    _angles_               = []
    _children_        = parent_to_children[_parent_]
    _children_labels_ = set()
    for _child_ in _children_:
        if _child_ >= len(pts): all_children_at_leaves = False
        else:                   
            _angles_.append(pts[int(_child_)])
            _children_labels_.add(clusterer.labels_[int(_child_)])
    if all_children_at_leaves:
        _angle_  = rt.averageDegrees(_angles_)
        _x_, _y_ = (w/2) + (r - 40) * cos(to_rad(_angle_)), (h/2) + (r - 40) * sin(to_rad(_angle_))
        if len(_children_labels_) == 1: _color_ = rt.co_mgr.getColor(list(_children_labels_)[0])
        else:                           _color_ = '#000000'
        svg_ring_one.append(f'<circle cx="{_x_}" cy="{_y_}" r="3" fill="{_color_}" />')
rt.tile([''.join(svg)+''.join(svg_ring_one)+'</svg>'])

In [ ]:
_parent_to_angles_ = {}
_parent_to_angle_  = {}
_node_depth_       = {}
_parent_completes_ = set()
for i in range(len(pts)): 
    _parent_completes_.add(i)
    _parent_to_angles_[i] = [pts[i][0]]
    _parent_to_angle_ [i] = pts[i][0]
def fillParentAngles(_parent_):
    if _parent_ in _parent_completes_: return
    _angles_ = []
    for _child_ in parent_to_children[_parent_]:
        if _child_ not in _parent_completes_: fillParentAngles(_child_)
        _angles_.append(_parent_to_angle_[_child_])
    _angle_                      = rt.averageDegrees(_angles_)
    _parent_to_angles_[_parent_] = _angles_
    _parent_to_angle_ [_parent_] = _angle_
_root_ = None
for _parent_ in parent_to_children.keys():
    _parent_ = int(_parent_)
    if _parent_ not in child_to_parent: 
        if _root_ is not None: raise Exception("multiple roots found!")
        _root_ = _parent_
    fillParentAngles(_parent_)
def calculateDepth(_node_):
    if _node_ in _node_depth_: return _node_depth_[_node_]
    if _node_ not in parent_to_children: _node_depth_[_node_] = 1
    else:                                _node_depth_[_node_] = 1 + max([calculateDepth(_child_) for _child_ in parent_to_children[_node_]])
    return _node_depth_[_node_]
tree_depth = calculateDepth(_root_)
svg_rings = []
def ringRadius(_depth_): return r - r * (_depth_-1)/(tree_depth-1)
for i in range(1, tree_depth+1):
    svg_rings.append(f'<circle cx="{w/2}" cy="{h/2}" r="{ringRadius(i)}" fill="none" stroke="#000000" stroke-width="0.1" />')
for _node_ in _node_depth_.keys():
    _depth_  = _node_depth_[_node_]
    _r_ring_ = ringRadius(_depth_)
    _angle_  = _parent_to_angle_[_node_]
    svg_rings.append(f'<circle cx="{(w/2) + _r_ring_ * cos(to_rad(_angle_))}" cy="{(h/2) + _r_ring_ * sin(to_rad(_angle_))}" r="2" fill="black" />')
svg_rings.append(f'<circle cx="{w/2}" cy="{h/2}" r="6" fill="none" stroke="#ff0000" stroke-width="0.5" />')
rt.tile([''.join(svg)+''.join(svg_rings)+'</svg>'])